In [1]:
import re
import pandas as pd
from typing import  Literal,List,Any
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.types import Command
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict, Annotated
from langchain_core.prompts.chat import ChatPromptTemplate
from langgraph.graph import START, StateGraph,END
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field, field_validator
from langchain_core.messages import HumanMessage,AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv



In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY= os.getenv("GROQ_API_KEY")
# print(GROQ_API_KEY)

In [3]:
chat_groq= ChatGroq(model= "llama-3.1-8b-instant")

In [4]:
chat_groq.invoke("hello,what is your name").content

"I'm an artificial intelligence model known as a conversational AI. I don't have a personal name, but I'm here to assist and communicate with you in a helpful and friendly way. You can think of me as a helpful chatbot or a conversational partner. How can I assist you today?"

In [5]:
class DateTimeModel(BaseModel):
    date: str = Field(description="Properly formatted date", pattern=r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$')

    @field_validator("date")
    def check_format_date(cls, v):
        if not re.match(r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$', v):  # Ensures 'DD-MM-YYYY HH:MM' format
            raise ValueError("The date should be in format 'DD-MM-YYYY HH:MM'")
        return v

In [6]:
class DateModel(BaseModel):
    date: str = Field(description="Properly formatted date", pattern=r'^\d{2}-\d{2}-\d{4}$')

    @field_validator("date")
    def check_format_date(cls, v):
        if not re.match(r'^\d{2}-\d{2}-\d{4}$', v):  # Ensures DD-MM-YYYY format
            raise ValueError("The date must be in the format 'DD-MM-YYYY'")
        return v

In [7]:

class IdentificationNumberModel(BaseModel):
    id: int = Field(description="Identification number (7 or 8 digits long)")
    @field_validator("id")
    def check_format_id(cls, v):
        if not re.match(r'^\d{7,8}$', str(v)):  # Convert to string before matching
            raise ValueError("The ID number should be a 7 or 8-digit number")
        return v

In [32]:
@tool
def check_availability_by_doctor(desired_date:DateModel,doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Checking the database if we have availability for the specific doctor.
    The parameters should be mentioned by the user in the query
    """
    df= pd.read_csv(r'C:\Users\yogass\Desktop\doctor_appointment_multiagent\notebook\availability.csv')
    #print(df)
    df['date_slot_time']= df['date_slot'].apply(lambda x:x.split(" ")[-1])
    rows = list(df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date)&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]['date_slot_time'])
    if len(rows)==0:
        output=  "NO availability of doctor"
    else:
        output= f'this is the availability of doctor in : {desired_date.date} \n'
        output+= "Availbale slots: "+", ".join(rows)
    return output

In [37]:
desired_date = DateModel(date="03-09-2024")

In [38]:
print(check_availability_by_doctor.invoke({
    "desired_date": desired_date,
    "doctor_name": 'john doe'
}))

this is the availability of doctor in : 03-09-2024 
Availbale slots: 08:30, 10:00, 10:30, 11:00, 11:30, 12:00, 13:00, 13:30, 14:00, 14:30, 15:00, 16:30


In [ ]:
@tool
def check_availability_by_specilization(desired_date: DateModel, specilization: Literal["general_dentist", "cosmetic_dentist", "prosthodontist", "pediatric_dentist","emergency_dentist","oral_surgeon","orthodontist"]):
    """
    Checking the database if we have availability for the specific specialization.
    The parameters should be mentioned by the user in the query
    """
    df= pd.read_csv(r"C:\Users\yogass\Desktop\doctor_appointment_multiagent\notebook\availability.csv")
    df['date_slot_time']= df['date_slot'].apply(lambda x:x.split(' '))[-1]
    rows = list(df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date)&(df['specialization'] == specilization)&(df['is_available'] == True)].groupby(['specialization', 'doctor_name'])['date_slot_time'])
    if len(rows)==0:
        output= "No availability for the desired date and specialization"


IndentationError: unindent does not match any outer indentation level (<string>, line 7)